# Task I - Data Preparation

This notebook performs data cleaning and initial exploration of the online retail dataset.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1. Load the Dataset

In [2]:
data_path = '../online_retail_II.xlsx'
df = pd.read_excel(data_path)

print(f"Dataset shape: {df.shape}")
print(f"\nColumn names and types:")
print(df.dtypes)
print(f"\nFirst few rows:")
df.head()

Dataset shape: (525461, 8)

Column names and types:
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

First few rows:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 2. Initial Data Overview

In [3]:
print("Dataset Info:")
print(df.info())
print(f"\nMissing values:")
print(df.isnull().sum())
print(f"\nDataset statistics:")
df.describe()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB
None

Missing values:
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Dataset statistics:


,Quantity,InvoiceDate,Price,Customer ID
count,525461.000000,525461,525461.000000,417534.000000
mean,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478
min,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000
25%,1.000000,2010-03-21 12:20:00,1.250000,13983.000000
50%,3.000000,2010-07-06 09:51:00,2.100000,15311.000000
75%,10.000000,2010-10-15 12:45:00,4.210000,16799.000000
max,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000
std,107.424110,NaN,146.126914,1680.811316


## 3. Data Cleaning

In [4]:
print(f"Original dataset size: {len(df)} rows")

df_clean = df.copy()

print(f"\n--- Removing Null Values ---")
print(f"Rows with null values before removal: {df_clean.isnull().sum().sum()}")
df_clean = df_clean.dropna()
print(f"Rows after removing nulls: {len(df_clean)}")

print(f"\n--- Removing Duplicate Rows ---")
print(f"Duplicate rows before removal: {df_clean.duplicated().sum()}")
df_clean = df_clean.drop_duplicates()
print(f"Rows after removing duplicates: {len(df_clean)}")

print(f"\n--- Handling Negative Values ---")
print(f"Negative prices: {(df_clean['Price'] < 0).sum()}")
print(f"Negative quantities: {(df_clean['Quantity'] < 0).sum()}")
print(f"Zero prices: {(df_clean['Price'] == 0).sum()}")
print(f"Zero quantities: {(df_clean['Quantity'] == 0).sum()}")

df_clean = df_clean[(df_clean['Price'] > 0) & (df_clean['Quantity'] > 0)]
print(f"\nRows after removing negative and zero values: {len(df_clean)}")

print(f"\nTotal rows removed: {len(df) - len(df_clean)}")
print(f"Final dataset size: {len(df_clean)} rows ({(len(df_clean)/len(df)*100):.2f}% of original)")

Original dataset size: 525461 rows

--- Removing Null Values ---
Rows with null values before removal: 110855
Rows after removing nulls: 417534

--- Removing Duplicate Rows ---
Duplicate rows before removal: 6771
Rows after removing duplicates: 410763

--- Handling Negative Values ---
Negative prices: 0
Negative quantities: 9816
Zero prices: 31
Zero quantities: 0

Rows after removing negative and zero values: 400916

Total rows removed: 124545
Final dataset size: 400916 rows (76.30% of original)


## 4. Analysis of Items (Products)

In [5]:
print(f"Total unique items (StockCode): {df_clean['StockCode'].nunique()}")
print(f"Total unique descriptions: {df_clean['Description'].nunique()}")

print(f"\n--- Item Statistics ---")
item_stats = df_clean.groupby('StockCode').agg({
    'Description': 'first',
    'Price': ['mean', 'min', 'max', 'std'],
    'Quantity': ['sum', 'mean', 'count'],
    'Invoice': 'nunique'
}).round(2)

item_stats.columns = ['Description', 'Avg_Price', 'Min_Price', 'Max_Price', 'Std_Price', 
                       'Total_Quantity', 'Avg_Quantity', 'Transaction_Count', 'Invoice_Count']

print(item_stats.head(10))

Total unique items (StockCode): 4017
Total unique descriptions: 4444

--- Item Statistics ---
                            Description  Avg_Price  Min_Price  Max_Price  \
StockCode                                                                  
10002       INFLATABLE POLITICAL GLOBE        0.84       0.72       0.85   
10080          GROOVY CACTUS INFLATABLE       0.85       0.85       0.85   
10109              BENDY COLOUR PENCILS       0.42       0.42       0.42   
10120                      DOGGY RUBBER       0.21       0.21       0.21   
10125           MINI FUNKY DESIGN TAPES       0.85       0.85       0.85   
10133      COLOURING PENCILS BROWN TUBE       0.85       0.72       0.85   
10134      COLOURING PENCILS BROWN TUBE       1.24       1.06       1.25   
10135      COLOURING PENCILS BROWN TUBE       1.24       1.06       1.25   
10138         ASSORTED COLOUR JUMBO PEN       0.42       0.42       0.42   
11001       ASSTD DESIGN RACING CAR PEN       1.66       0.50       1.

In [6]:
print("Top 10 Items by Total Quantity Sold:")
top_items_quantity = df_clean.groupby('StockCode').agg({
    'Description': 'first',
    'Quantity': 'sum',
    'Price': 'mean',
    'Invoice': 'count'
}).sort_values('Quantity', ascending=False).head(10)

top_items_quantity.columns = ['Description', 'Total_Quantity', 'Avg_Price', 'Transactions']
print(top_items_quantity)

print("\n\nTop 10 Items by Average Price:")
top_items_price = df_clean.groupby('StockCode').agg({
    'Description': 'first',
    'Price': 'mean',
    'Quantity': 'sum',
    'Invoice': 'count'
}).sort_values('Price', ascending=False).head(10)

top_items_price.columns = ['Description', 'Avg_Price', 'Total_Quantity', 'Transactions']
print(top_items_price)

Top 10 Items by Total Quantity Sold:
                                  Description  Total_Quantity  Avg_Price  \
StockCode                                                                  
21212        PACK OF 72 RETRO SPOT CAKE CASES           57505   0.536143   
85123A     WHITE HANGING HEART T-LIGHT HOLDER           56814   2.854422   
84077       WORLD WAR 2 GLIDERS ASSTD DESIGNS           54610   0.231104   
85099B            JUMBO BAG RED WHITE SPOTTY            48731   1.907341   
17003                     BROCADE RING PURSE            48166   0.203646   
84879           ASSORTED COLOUR BIRD ORNAMENT           44431   1.671697   
84991             60 TEATIME FAIRY CAKE CASES           35656   0.540539   
21977      PACK OF 60 PINK PAISLEY CAKE CASES           30871   0.540222   
22197                 POPCORN HOLDER , SMALL            28773   0.837301   
37410      BLACK AND WHITE PAISLEY FLOWER MUG           25685   1.045000   

           Transactions  
StockCode               

## 5. Customer and Geographic Analysis

In [7]:
print(f"Total unique customers: {df_clean['Customer ID'].nunique()}")
print(f"Total unique countries: {df_clean['Country'].nunique()}")

print(f"\nTop 10 Countries by Transaction Count:")
country_transactions = df_clean['Country'].value_counts().head(10)
print(country_transactions)

print(f"\nTop 10 Countries by Total Quantity:")
country_quantity = df_clean.groupby('Country')['Quantity'].sum().sort_values(ascending=False).head(10)
print(country_quantity)

print(f"\nTop 10 Customers by Total Spending:")
df_clean['Total_Spending'] = df_clean['Price'] * df_clean['Quantity']
customer_spending = df_clean.groupby('Customer ID')['Total_Spending'].sum().sort_values(ascending=False).head(10)
print(customer_spending)

Total unique customers: 4312
Total unique countries: 37

Top 10 Countries by Transaction Count:
Country
United Kingdom    364233
EIRE                8503
Germany             7645
France              5452
Netherlands         2728
Spain               1228
Switzerland         1170
Belgium             1036
Portugal             983
Sweden               867
Name: count, dtype: int64

Top 10 Countries by Total Quantity:
Country
United Kingdom    4430148
Denmark            229690
Netherlands        183615
EIRE               181413
France             162048
Germany            108615
Sweden              52417
Spain               22841
Switzerland         22255
Australia           20189
Name: Quantity, dtype: int64

Top 10 Customers by Total Spending:
Customer ID
18102.0    349164.35
14646.0    248396.50
14156.0    196549.74
14911.0    152121.22
13694.0    131443.19
17511.0     84541.17
15061.0     83284.38
16684.0     80489.21
16754.0     65500.07
17949.0     60117.60
Name: Total_Spending, dtype

## 6. Price and Quantity Distribution

In [8]:
print("Price Statistics:")
print(df_clean['Price'].describe().round(2))

print("\nQuantity Statistics:")
print(df_clean['Quantity'].describe().round(2))

print("\nTransaction Value (Price x Quantity) Statistics:")
df_clean['Transaction_Value'] = df_clean['Price'] * df_clean['Quantity']
print(df_clean['Transaction_Value'].describe().round(2))

Price Statistics:
count    400916.00
mean          3.31
std          35.05
min           0.00
25%           1.25
50%           1.95
75%           3.75
max       10953.50
Name: Price, dtype: float64

Quantity Statistics:
count    400916.00
mean         13.77
std          97.64
min           1.00
25%           2.00
50%           5.00
75%          12.00
max       19152.00
Name: Quantity, dtype: float64

Transaction Value (Price x Quantity) Statistics:
count    400916.00
mean         21.95
std          77.76
min           0.00
25%           5.00
50%          12.50
75%          19.50
max       15818.40
Name: Transaction_Value, dtype: float64


## 7. Summary and Data Quality Report

In [9]:
summary_report = {
    'Original Rows': len(df),
    'Cleaned Rows': len(df_clean),
    'Rows Removed': len(df) - len(df_clean),
    'Removal Percentage': f"{((len(df) - len(df_clean))/len(df)*100):.2f}%",
    'Unique Products': df_clean['StockCode'].nunique(),
    'Unique Customers': df_clean['Customer ID'].nunique(),
    'Countries': df_clean['Country'].nunique(),
    'Date Range': f"{df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}",
    'Avg Price': f"${df_clean['Price'].mean():.2f}",
    'Avg Quantity': f"{df_clean['Quantity'].mean():.2f}",
    'Total Revenue': f"${df_clean['Transaction_Value'].sum():.2f}"
}

print("\n" + "="*50)
print("DATA PREPARATION SUMMARY")
print("="*50)
for key, value in summary_report.items():
    print(f"{key}: {value}")
print("="*50)


DATA PREPARATION SUMMARY
Original Rows: 525461
Cleaned Rows: 400916
Rows Removed: 124545
Removal Percentage: 23.70%
Unique Products: 4017
Unique Customers: 4312
Countries: 37
Date Range: 2009-12-01 07:45:00 to 2010-12-09 20:01:00
Avg Price: $3.31
Avg Quantity: 13.77
Total Revenue: $8798233.74


## 8. Export Cleaned Data

In [10]:
df_clean.to_csv('cleaned_data.csv', index=False)
print("Cleaned data exported to 'cleaned_data.csv'")
file_size = df_clean.memory_usage(deep=True).sum() / (1024**2)
print("File size:", round(file_size, 2), "MB")

Cleaned data exported to 'cleaned_data.csv'
File size: 102.75 MB
